# Train Llama3-8b for a sensationalism detector using unsloth


Run training script: 

jupyter nbconvert --to notebook --execute train_llm.ipynb


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

### Load model in 4bit quantization

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",   
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, )

## Prompt for instruction tuning
    The prompt contains detailed instructions for making the LLM perform a classification task and also includes our definition of sensationalist.

In [ ]:
prompt = """ 
Below you will find the title, summary, and body of a news article. 
Your task is to analyze these components and classify whether the article is sensationalist or not.

Sensationalist is defined as: "presenting information in a way that is intended to provoke public interest, excitement, or anxiety, at the expense of accuracy."

### Article information:
    Title: {}
    Subheading: {}
    Body: {}
    is this article fake-news?: {}
    
### is this article sensationalist?:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    
    title = examples["Titular"]
    summary = examples["Copete"]
    body      = examples["Cuerpo"]
    is_false = examples["Falsa"]
    is_sensationalist      = examples["Amarillismo"]
    
    texts = []
    for title, summary, body, is_false, is_sensationalist in zip(title, summary, body, is_false, is_sensationalist):
        # Add EOS_TOKEN
        text = prompt.format(title, summary, body, is_false, is_sensationalist) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

### Load train dataset 

In [ ]:
import pandas as pd
from datasets import Dataset

sensationalist_dataset = pd.read_json("data/trainset.json", orient='index',  encoding='latin1')

#Build prompts using trainset
dataset_dir = formatting_prompts_func(sensationalist_dataset)
dataset = Dataset.from_dict(dataset_dir)

## Define trainer parameters 

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 5000,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "llama-3-8b-bnb-4bit-sft-sensationalist-detector-v2",
    ),
)

## Train 

In [ ]:
trainer_stats = trainer.train()

# Save weights
model.save_pretrained("llama-3-8b-bnb-4bit-sft-sensationalist-detector-v2_model")